In [1]:
!pip install keras
!pip install tensorflow

In [ ]:
# adapted from https://github.com/Conferences2023/TPDL/blob/main/Citation%20Analysis/LSTM.py
'''code of Usman, M., & Balke, W.-T. (2023). On retraction cascade? Citation intention analysis as a quality control mechanism in digital libraries. In O. Alonso, H. Cousijn, G. Silvello, M. Marrero, C. Teixeira Lopes, & S. Marchesin (Eds.), Linking Theory and Practice of Digital Libraries (pp. 117–131). Springer Nature Switzerland. https://doi.org/10.1007/978-3-031-43849-3_11'''

In [2]:
import pandas as pd
import numpy as np
import openpyxl
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import keras
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import confusion_matrix, classification_report
import pandas as pd

In [3]:
# Load data from CSV file
train_data = pd.read_csv('ML_data/train_set.csv')
test_data = pd.read_csv('ML_data/test_set.csv')

In [5]:
# Encode labels
le = LabelEncoder()
train_labels = le.fit_transform(train_data['Citation context level annotation'])
test_labels = le.transform(test_data['Citation context level annotation'])

# Tokenize citation context text
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data['Citation context (sent)'])
train_sequences = tokenizer.texts_to_sequences(train_data['Citation context (sent)'])
test_sequences = tokenizer.texts_to_sequences(test_data['Citation context (sent)'])

# Pad sequences to same length
max_len = 100
train_sequences = pad_sequences(train_sequences, maxlen=max_len)
test_sequences = pad_sequences(test_sequences, maxlen=max_len)

# Define neural network architecture
model = Sequential()
model.add(Embedding(5000, 128, input_length=max_len))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train model
# model.fit(train_sequences, train_labels, epochs=30, batch_size=32, validation_data=(test_sequences, test_labels))
model.fit(train_sequences, train_labels, epochs=30, batch_size=32)

# Make predictions on test set
y_pred = model.predict(test_sequences)
y_pred = (y_pred > 0.5)

# Print classification report
print(classification_report(test_labels, y_pred, target_names=['N', 'Y']))

# Print confusion matrix
print(confusion_matrix(test_labels, y_pred))

Epoch 1/30


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


8/8 ━━━━━━━━━━━━━━━━━━━━ 8s 207ms/step - accuracy: 0.6134 - loss: 0.6864
Epoch 2/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 159ms/step - accuracy: 0.6666 - loss: 0.6443
Epoch 3/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 160ms/step - accuracy: 0.6486 - loss: 0.6230
Epoch 4/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 161ms/step - accuracy: 0.6778 - loss: 0.5721
Epoch 5/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 162ms/step - accuracy: 0.7303 - loss: 0.4666
Epoch 6/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 279ms/step - accuracy: 0.9513 - loss: 0.2738
Epoch 7/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 159ms/step - accuracy: 0.9621 - loss: 0.1975
Epoch 8/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 161ms/step - accuracy: 0.9853 - loss: 0.1068
Epoch 9/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 162ms/step - accuracy: 0.9797 - loss: 0.0963
Epoch 10/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 165ms/step - accuracy: 0.9667 - loss: 0.0895
Epoch 11/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 171ms/step - accuracy: 0.9861 - loss: 0.0764
Epoch 12/30
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 235ms/step - accuracy: 0.9963 - loss: 0.0296


In [10]:
# create predictions for all instances
full_data = pd.read_csv('data_all.csv')
full_sequences = tokenizer.texts_to_sequences(full_data['Citation context (sent)'])
full_sequences = pad_sequences(full_sequences, maxlen=max_len)
full_predictions = model.predict(full_sequences)
full_predictions = (full_predictions > 0.5)
full_data['Predicted label'] = full_predictions
full_data.to_csv('data/tilo_LSTM_pred_203.csv', index=False)

10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 67ms/step
